In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
sns.set(color_codes=True)

In [2]:
data = pd.read_csv('./RawDatafromJim_Complete.csv', usecols=[0,1,3,6,7,16])
data = data.apply(lambda x: pd.to_numeric(x, errors='ignore'))
data["weight"]=1

In [3]:
#build case year lookup
case_year_dict ={}
citing_year = data.filter(['citing_case','citing_case_year'], axis=1)
citing_year = citing_year.rename(columns={'citing_case':'case', 'citing_case_year': 'year'})
cited_year = data.filter(['cited_case','cited_case_year'], axis=1)
cited_year = cited_year.rename(columns={'cited_case':'case', 'cited_case_year': 'year'})
case_year = citing_year.append(cited_year)
#create lookup for case and year
case_year_dict = {}
for row in case_year.iterrows():
    case_year_dict[row[1].case] = row[1].year

In [4]:
#build case year opinion type
case_opinion_type_dict ={}
citing_year = data.filter(['citing_case','citing_opinion_type'], axis=1)
for row in citing_year.iterrows():
    case_opinion_type_dict[row[1].citing_case] = row[1].citing_opinion_type

In [5]:
#build case year opinion type
case_usid_dict ={}
citing_year = data.filter(['cited_case','cited_usid'], axis=1)
for row in citing_year.iterrows():
    case_usid_dict[row[1].cited_case] = row[1].cited_usid

In [7]:
def case_data_for_year(calc_year):
    #limit cases to based in year or before
    year_data = data.loc[data['citing_case_year'] <= calc_year]
    #build network
    G = nx.from_pandas_dataframe(year_data,"citing_case","cited_case",["weight"],nx.DiGraph())
    #calculate hub and authority
    h,a = nx.hits(G,1000,normalized=False)
    if(len(h) > 0 and len(a) > 0):
        d = []
        for key in h:
            d.append({'calc_year': calc_year, 'case':key, 'case_year': case_year_dict[key], 'opinion_type': case_opinion_type_dict[key], 'case_id': case_usid_dict[key], 'hub_raw': h[key], 'auth_raw': a[key] })
        year_case_scores = pd.DataFrame(d)
        year_case_scores['hub_percentile'] = year_case_scores.rank(pct=True)['hub_raw']
        year_case_scores['hub_rank'] = year_case_scores["hub_raw"].rank(ascending=False)
        year_case_scores['auth_percentile'] = year_case_scores.rank(pct=True)['auth_raw']
        year_case_scores['auth_rank'] = year_case_scores["auth_raw"].rank(ascending=False)
        
    else:
        return None
    return year_case_scores


In [12]:
#years_to_analyize = data.cited_case_year.unique()
#years_to_analyize.sort()
#years_to_analyize = years_to_analyize[0:10]
years_to_analyize = [1800]

In [13]:
case_data = pd.DataFrame()
for year in years_to_analyize:
    cdy = case_data_for_year(year)
    if(cdy is None):
        continue
    else:
        case_data = case_data.append(cdy)

KeyError: '1796 U.S. LEXIS 409'

In [11]:
#case_data.sort_values(by="hub_rank",inplace=True)
case_data

""
